In [2]:
!pip install pandas lightgbm optuna scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 12.0 MB/s eta 0:00:00


In [3]:
!pip install xgboost

In [4]:
import pandas as pd
from lightgbm import LGBMRanker
from xgboost import XGBRanker
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score
import optuna
import numpy as np

In [5]:
df = pd.read_csv('intern_task.csv')
print(df.info())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235258 entries, 0 to 235257
Columns: 146 entries, rank to feature_143
dtypes: float64(140), int64(6)
memory usage: 262.1 MB
None
                rank       query_id      feature_0      feature_1  \
count  235258.000000  235258.000000  235258.000000  235258.000000   
mean        0.677869   14828.413401       1.911960       0.206233   
std         0.830033    8193.945170       1.237374       0.579089   
min         0.000000      10.000000       0.000000       0.000000   
25%         0.000000    8215.000000       1.000000       0.000000   
50%         0.000000   14935.000000       2.000000       0.000000   
75%         1.000000   21580.000000       3.000000       0.000000   
max         4.000000   29995.000000      31.000000      18.000000   

           feature_2      feature_3      feature_4      feature_5  \
count  235258.000000  235258.000000  235258.000000  235258.000000   
mean        1.189847       0.550272       1.960082       0.80

In [6]:
X = df.drop(['rank'], axis=1)
y = df['rank']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
def ndcg_at_k(y_true, y_score, query_ids, k=5):
    def dcg_at_k(r, k):
        r = np.asfarray(r)[:k]
        if r.size:
            return np.sum((2 ** r - 1) / np.log2(np.arange(2, r.size + 2)))
        return 0.

    def ndcg_at_k(r, k):
        idcg = dcg_at_k(sorted(r, reverse=True), k)
        if not idcg:
            return 0.
        return dcg_at_k(r, k) / idcg

    ndcg_scores = []
    unique_queries = np.unique(query_ids)

    for query in unique_queries:
        query_indices = np.where(query_ids == query)[0]
        y_true_query = y_true[query_indices]
        y_score_query = y_score[query_indices]

        order = np.argsort(y_score_query)[::-1]
        y_true_query_sorted = y_true_query[order]

        ndcg_scores.append(ndcg_at_k(y_true_query_sorted, k))

    return np.mean(ndcg_scores)

In [ ]:
def objective_lgbm(trial):
    num_leaves = trial.suggest_int('num_leaves', 10, 100)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    num_boost_round = trial.suggest_int('num_boost_round', 50, 500)
    min_child_samples = trial.suggest_int('min_child_samples', 5, 50)
    reg_alpha = trial.suggest_float('reg_alpha', 0.0, 1.0)
    reg_lambda = trial.suggest_float('reg_lambda', 0.0, 1.0)

    model = LGBMRanker(objective='lambdarank',
                       metric='ndcg',
                       num_leaves=num_leaves,
                       max_depth=max_depth,
                       learning_rate=learning_rate,
                       n_estimators=num_boost_round,
                       min_child_samples=min_child_samples,
                       reg_alpha=reg_alpha,
                       reg_lambda=reg_lambda,
                       random_state=42
                       )

    model.fit(X_train, y_train, group=X_train.groupby('query_id').size().values)
    y_pred = model.predict(X_test)
    query_ids = X_test['query_id'].values
    ndcg_5 = ndcg_at_k(y_test.values, y_pred, query_ids, k=5)
    return ndcg_5

Обучим сначлаа LGBMRanker, XGBRanker на ненормализованных данных, посмотрим, что они скажут

In [ ]:
study_lgbm = optuna.create_study(direction='maximize')
study_lgbm.optimize(objective_lgbm, n_trials=50)

[I 2024-04-27 14:21:54,701] A new study created in memory with name: no-name-1067ae76-3843-45b4-8a36-0651967af0e2


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.368635 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26698
[LightGBM] [Info] Number of data points in the train set: 188206, number of used features: 141
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

[I 2024-04-27 14:22:52,079] Trial 0 finished with value: 0.5167747782588693 and parameters: {'num_leaves': 15, 'max_depth': 4, 'learning_rate': 0.18525619726611026, 'num_boost_round': 311, 'min_child_samples': 27, 'reg_alpha': 0.9448999441121265, 'reg_lambda': 0.5175316641534804}. Best is trial 0 with value: 0.5167747782588693.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.148356 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 26698
[LightGBM] [Info] Number of data points in the train set: 188206, number of used features: 141
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with 

[I 2024-04-27 14:24:38,792] Trial 1 finished with value: 0.5306737911337334 and parameters: {'num_leaves': 63, 'max_depth': 7, 'learning_rate': 0.2624605627029138, 'num_boost_round': 485, 'min_child_samples': 23, 'reg_alpha': 0.23903936646682444, 'reg_lambda': 0.31185185877510113}. Best is trial 1 with value: 0.5306737911337334.


...

[I 2024-04-27 15:26:55,616] Trial 47 finished with value: 0.5272392515009529 and parameters: {'num_leaves': 57, 'max_depth': 10, 'learning_rate': 0.20513962071492475, 'num_boost_round': 315, 'min_child_samples': 47, 'reg_alpha': 0.6816134705138497, 'reg_lambda': 0.4727005183227181}. Best is trial 25 with value: 0.5373661001696393.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.158483 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 26698
[LightGBM] [Info] Number of data points in the train set: 188206, number of used features: 141
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2024-04-27 15:28:53,260] Trial 48 finished with value: 0.5354594584245931 and parameters: {'num_leaves': 95, 'max_depth': 9, 'learning_rate': 0.1786159953122448, 'num_boost_round': 442, 'min_child_samples': 26, 'reg_alpha': 0.7332048882134844, 'reg_lambda': 0.6649684767752829}. Best is trial 25 with value: 0.5373661001696393.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.409428 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26698
[LightGBM] [Info] Number of data points in the train set: 188206, number of used features: 141
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2024-04-27 15:30:22,992] Trial 49 finished with value: 0.5302793613556971 and parameters: {'num_leaves': 95, 'max_depth': 9, 'learning_rate': 0.17864234520098377, 'num_boost_round': 284, 'min_child_samples': 28, 'reg_alpha': 0.740738135171739, 'reg_lambda': 0.8409025683743185}. Best is trial 25 with value: 0.5373661001696393.


In [ ]:
print(f'Best NDCG@5 for LGBM: {study_lgbm.best_value:.4f}')
print('Best hyperparameters for LGBM:')
print(study_lgbm.best_params)

Best NDCG@5 for LGBM: 0.5374
Best hyperparameters for LGBM:
{'num_leaves': 87, 'max_depth': 9, 'learning_rate': 0.10264292610732471, 'num_boost_round': 336, 'min_child_samples': 35, 'reg_alpha': 0.14067762968187958, 'reg_lambda': 0.7409153013004788}


Best NDCG@5 for LGBM: 0.5374
Best hyperparameters for LGBM:
{'num_leaves': 87, 'max_depth': 9, 'learning_rate': 0.10264292610732471, 'num_boost_round': 336, 'min_child_samples': 35, 'reg_alpha': 0.14067762968187958, 'reg_lambda': 0.7409153013004788}

In [8]:
def objective_xgb(trial):
    params = {
        'objective': 'rank:ndcg',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.0, 1.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0)
    }

    model = XGBRanker(**params)
    model.fit(X_train, y_train, group=X_train.groupby('query_id').size().values)

    y_pred = model.predict(X_test)
    query_ids = X_test['query_id'].values
    ndcg_5 = ndcg_at_k(y_test.values, y_pred, query_ids, k=5)
    return ndcg_5

In [9]:
study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=50)

[I 2024-04-28 09:52:21,795] A new study created in memory with name: no-name-414bd940-2a38-42b8-9fb9-21057be1dbc3
[I 2024-04-28 09:54:02,750] Trial 0 finished with value: 0.5254054283688235 and parameters: {'learning_rate': 0.26218993026078813, 'max_depth': 10, 'n_estimators': 180, 'min_child_weight': 2, 'gamma': 0.08914924162477034, 'subsample': 0.5765190206098445, 'colsample_bytree': 0.8345556118632947, 'reg_alpha': 0.24865980102439433, 'reg_lambda': 0.4723289875435238}. Best is trial 0 with value: 0.5254054283688235.
[I 2024-04-28 09:54:57,253] Trial 1 finished with value: 0.5222489861015802 and parameters: {'learning_rate': 0.14686811331094282, 'max_depth': 9, 'n_estimators': 110, 'min_child_weight': 3, 'gamma': 0.17586310610448486, 'subsample': 0.9763738856040041, 'colsample_bytree': 0.9206445495342952, 'reg_alpha': 0.3680765008215475, 'reg_lambda': 0.937521181993417}. Best is trial 0 with value: 0.5254054283688235.
[I 2024-04-28 09:56:00,961] Trial 2 finished with value: 0.514947

In [10]:
print(f'Best NDCG@5 for XGB: {study_xgb.best_value:.4f}')
print('Best hyperparameters for XGB:')
print(study_xgb.best_params)

Best NDCG@5 for XGB: 0.5436
Best hyperparameters for XGB:
{'learning_rate': 0.18355585767238483, 'max_depth': 9, 'n_estimators': 456, 'min_child_weight': 2, 'gamma': 0.007183842663282607, 'subsample': 0.9619329921087644, 'colsample_bytree': 0.5487403237135628, 'reg_alpha': 0.46590326161117473, 'reg_lambda': 0.05390754790505921}


Best NDCG@5 for XGB: 0.5436
Best hyperparameters for XGB:
{'learning_rate': 0.18355585767238483, 'max_depth': 9, 'n_estimators': 456, 'min_child_weight': 2, 'gamma': 0.007183842663282607, 'subsample': 0.9619329921087644, 'colsample_bytree': 0.5487403237135628, 'reg_alpha': 0.46590326161117473, 'reg_lambda': 0.05390754790505921}


Теперь отнормализуем данные с помощью StandartScaler из sklearn`a и посмотрим, что получится!

In [11]:
from sklearn.preprocessing import StandardScaler

In [12]:
scaler = StandardScaler()
columns = list(X.columns)
columns.remove('query_id')
X_new = X.copy()
X_new[columns] = scaler.fit_transform(X_new[columns])

X_train_normalized, X_test_normalized, y_train_normalized, y_test_normalized = train_test_split(X_new, y, test_size=0.2, random_state=42)

In [13]:
pd.DataFrame(X_train_normalized).head()

,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143
154012,19060,-0.737015,1.370720,-0.183032,0.568595,-0.797721,0.576653,3.392204,1.092516,0.535736,...,-0.024367,-0.003950,0.543102,0.734952,-0.171766,1.831865,1.331198,-0.128656,-0.022884,0.797334
169995,20845,-0.737015,-0.356135,-0.183032,-0.695715,-0.797721,0.576653,-0.366710,1.092516,0.771081,...,-0.023651,-0.003944,-0.697956,0.734952,-0.141470,-0.718582,3.091526,-0.128656,-0.069974,0.588236
147302,18325,-0.737015,-0.356135,-0.183032,0.568595,0.033167,-0.894132,-0.366710,-0.098040,0.436933,...,-0.024367,-0.003950,0.405205,-0.913423,-0.145082,0.556642,-0.649485,-0.118120,-0.148457,-0.213306
187614,22585,-0.737015,-0.356135,-0.183032,0.568595,-0.797721,0.576653,-0.366710,1.092516,1.030515,...,-0.024367,-0.003950,0.405205,0.734952,-0.104076,1.831865,0.162974,-0.128656,-0.305424,-0.457254
140275,17650,0.071150,-0.356135,0.781073,-0.695715,0.033167,0.576653,-0.366710,1.092516,0.333463,...,-0.024367,-0.003950,-0.697956,0.621786,1.165303,-0.718582,0.339403,-0.033825,0.008509,0.135190


In [11]:
def objective_lgbm_normalized(trial):
    num_leaves = trial.suggest_int('num_leaves', 10, 100)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    num_boost_round = trial.suggest_int('num_boost_round', 50, 500)
    min_child_samples = trial.suggest_int('min_child_samples', 5, 50)
    reg_alpha = trial.suggest_float('reg_alpha', 0.0, 1.0)
    reg_lambda = trial.suggest_float('reg_lambda', 0.0, 1.0)

    model = LGBMRanker(objective='lambdarank',
                       metric='ndcg',
                       num_leaves=num_leaves,
                       max_depth=max_depth,
                       learning_rate=learning_rate,
                       n_estimators=num_boost_round,
                       min_child_samples=min_child_samples,
                       reg_alpha=reg_alpha,
                       reg_lambda=reg_lambda,
                       random_state=42
                       )

    model.fit(X_train_normalized, y_train_normalized, group=X_train_normalized.groupby('query_id').size().values)
    y_pred = model.predict(X_test_normalized)
    query_ids = X_test_normalized['query_id'].values
    ndcg_5 = ndcg_at_k(y_test_normalized.values, y_pred, query_ids, k=5)
    return ndcg_5

In [12]:
study_lgbm_normalized = optuna.create_study(direction='maximize')
study_lgbm_normalized.optimize(objective_lgbm_normalized, n_trials=50)

[I 2024-04-27 22:34:35,695] A new study created in memory with name: no-name-7746b2f9-84f1-45f0-9502-235060491ace


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.210827 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25872
[LightGBM] [Info] Number of data points in the train set: 58507, number of used features: 141


[I 2024-04-27 22:35:33,852] Trial 0 finished with value: 0.544856424582774 and parameters: {'num_leaves': 58, 'max_depth': 9, 'learning_rate': 0.01910694518090675, 'num_boost_round': 423, 'min_child_samples': 25, 'reg_alpha': 0.7193930934220838, 'reg_lambda': 0.7727070671761667}. Best is trial 0 with value: 0.544856424582774.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.104960 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25872
[LightGBM] [Info] Number of data points in the train set: 58507, number of used features: 141
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2024-04-27 22:36:13,899] Trial 1 finished with value: 0.5360465801991756 and parameters: {'num_leaves': 52, 'max_depth': 9, 'learning_rate': 0.18437705433451143, 'num_boost_round': 374, 'min_child_samples': 44, 'reg_alpha': 0.7308636934967967, 'reg_lambda': 0.25868612763739973}. Best is trial 0 with value: 0.544856424582774.


...

[I 2024-04-27 22:59:48,766] Trial 47 finished with value: 0.5504494240515068 and parameters: {'num_leaves': 91, 'max_depth': 10, 'learning_rate': 0.16756495455413667, 'num_boost_round': 344, 'min_child_samples': 38, 'reg_alpha': 0.7888350092652494, 'reg_lambda': 0.17245791715393474}. Best is trial 28 with value: 0.5516560577655624.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.121049 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25872
[LightGBM] [Info] Number of data points in the train set: 58507, number of used features: 141
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2024-04-27 23:00:33,455] Trial 48 finished with value: 0.5338605109093181 and parameters: {'num_leaves': 94, 'max_depth': 10, 'learning_rate': 0.21259970794275138, 'num_boost_round': 349, 'min_child_samples': 34, 'reg_alpha': 0.7974543020528618, 'reg_lambda': 0.2392664932624018}. Best is trial 28 with value: 0.5516560577655624.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.168675 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25872
[LightGBM] [Info] Number of data points in the train set: 58507, number of used features: 141
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2024-04-27 23:01:17,268] Trial 49 finished with value: 0.5374458778404759 and parameters: {'num_leaves': 94, 'max_depth': 10, 'learning_rate': 0.17144073535928478, 'num_boost_round': 314, 'min_child_samples': 37, 'reg_alpha': 0.6565999288078915, 'reg_lambda': 0.09311450899602369}. Best is trial 28 with value: 0.5516560577655624.


In [13]:
print(f'Best NDCG@5 for LGBM on normalized data: {study_lgbm_normalized.best_value:.4f}')
print('Best hyperparameters for LGBM on normalized data:')
print(study_lgbm_normalized.best_params)

Best NDCG@5 for LGBM on normalized data: 0.5517
Best hyperparameters for LGBM on normalized data:
{'num_leaves': 49, 'max_depth': 10, 'learning_rate': 0.10523945951163613, 'num_boost_round': 245, 'min_child_samples': 46, 'reg_alpha': 0.14230649990869138, 'reg_lambda': 0.13435149240608613}


Best NDCG@5 for LGBM on normalized data: 0.5517
Best hyperparameters for LGBM on normalized data:
{'num_leaves': 49, 'max_depth': 10, 'learning_rate': 0.10523945951163613, 'num_boost_round': 245, 'min_child_samples': 46, 'reg_alpha': 0.14230649990869138, 'reg_lambda': 0.13435149240608613}

In [14]:
def objective_xgb_normalized(trial):
    params = {
        'objective': 'rank:ndcg',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.0, 1.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0)
    }

    model = XGBRanker(**params)
    model.fit(X_train_normalized, y_train_normalized, group=X_train_normalized.groupby('query_id').size().values)
    y_pred = model.predict(X_test_normalized)
    query_ids = X_test_normalized['query_id'].values
    ndcg_5 = ndcg_at_k(y_test_normalized.values, y_pred, query_ids, k=5)
    return ndcg_5

In [16]:
study_xgb_normalized = optuna.create_study(direction='maximize')
study_xgb_normalized.optimize(objective_xgb_normalized, n_trials=50)

[I 2024-04-28 11:38:25,585] A new study created in memory with name: no-name-9360fce8-f79f-433a-a0aa-aa1f65ec2c73
[I 2024-04-28 11:40:47,194] Trial 0 finished with value: 0.5291498542358632 and parameters: {'learning_rate': 0.2416446345597921, 'max_depth': 6, 'n_estimators': 443, 'min_child_weight': 5, 'gamma': 0.08543194287951605, 'subsample': 0.7170540793713815, 'colsample_bytree': 0.8338159367745881, 'reg_alpha': 0.32619438634609565, 'reg_lambda': 0.7233550795809859}. Best is trial 0 with value: 0.5291498542358632.
[I 2024-04-28 11:42:17,823] Trial 1 finished with value: 0.5242759834510562 and parameters: {'learning_rate': 0.17815058696489794, 'max_depth': 6, 'n_estimators': 266, 'min_child_weight': 2, 'gamma': 0.196967866527347, 'subsample': 0.8627047382567901, 'colsample_bytree': 0.9767673050516534, 'reg_alpha': 0.5903722221875336, 'reg_lambda': 0.46151380119149377}. Best is trial 0 with value: 0.5291498542358632.
[I 2024-04-28 11:44:26,674] Trial 2 finished with value: 0.50488235

In [17]:
print(f'Best NDCG@5 for XGB on normalized data: {study_xgb_normalized.best_value:.4f}')
print('Best hyperparameters for XGB on normalized data:')
print(study_xgb_normalized.best_params)

Best NDCG@5 for XGB on normalized data: 0.5416
Best hyperparameters for XGB on normalized data:
{'learning_rate': 0.13541471418335874, 'max_depth': 10, 'n_estimators': 475, 'min_child_weight': 8, 'gamma': 0.055597247241479325, 'subsample': 0.7993063842562708, 'colsample_bytree': 0.8843911166501889, 'reg_alpha': 0.6321697450018331, 'reg_lambda': 0.4244566664938659}


Best NDCG@5 for XGB on normalized data: 0.5416
Best hyperparameters for XGB on normalized data:
{'learning_rate': 0.13541471418335874, 'max_depth': 10, 'n_estimators': 475, 'min_child_weight': 8, 'gamma': 0.055597247241479325, 'subsample': 0.7993063842562708, 'colsample_bytree': 0.8843911166501889, 'reg_alpha': 0.6321697450018331, 'reg_lambda': 0.4244566664938659}